# Adding Dependeces

In [69]:
using DelimitedFiles
using CUDA
using ProgressMeter
include("../init/forces.jl")
include("../init/Neighbor_CUDA.jl")

knn_cu (generic function with 1 method)

# Initial Parameters

In [70]:
# Physical Conditions
const R_agg = 10
const t_f = 10000

# Time model Conditions
dt = 0.1
n_knn = 2

# Constants for Force Model
r_max, s = 3, 1.9
K = 1
nn = 16

# Initial Coordinates of Aggregates
global X = Float32.(readdlm("../data/Initial/XYZ/Init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

746×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  2.5   0.58  -8.98
 -2.5   2.31  -8.98
 -0.5   2.31  -8.98
  ⋮           
  3.5  -2.31   8.98
 -3.5  -0.58   8.98
 -1.5  -0.58   8.98
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

In [71]:
X = Matrix(X)
X_f = zeros(size(X,1))
for i in 1:size(X_f,1)
    A = sqrt(X[i,1]^2+X[i,2]^2+X[i,3]^2)
    if A > 0.8*R_agg
        X_f[i] = 1
    else
        X_f[i] = 2
    end
end

X_f_2 = Int.(vcat(X_f,X_f))

  0.038437 seconds (88.25 k allocations: 6.229 MiB, 98.11% compilation time)


Process(`paraview Test_Zero.xyz`, ProcessExited(0))

# Functions

In [72]:
function init()
    # Definig Variables for calculing initial variables
    global X

    # Inizializate Variables for kNN
    global i_Cell = CuArray{Float32}(undef, (size(X, 1), size(X, 1), 3))
    global Dist = CuArray{Float32}(undef, (size(X, 1), size(X, 1)))
    global idx = hcat([[CartesianIndex(i,1) for i=1:nn] for j=1:size(X,1)]...) |> cu

    # Inizializate Variables for Forces
    global r = zeros(nn,size(X)[1],3) |> cu
    global dist = zeros(nn, size(X)[1]) |> cu
    global F = zeros(nn, size(X)[1],3) |> cu
    global dX = zeros(size(X)[1],3) |> cu;
end

init (generic function with 1 method)

In [73]:
function main(n_knn)
    println("------------Init()-----------")
    init()

    p = Progress(Int(t_f/dt),barlen=25)
    for i in 0:Int(t_f/dt)

        if mod(i, n_knn) == 0
            # Calculating kNN
            knn_cu()
        end

        # Calculating Forces
        forces_cu()

        global X = X + dX*dt
        next!(p)
    end
end

main (generic function with 1 method)

In [75]:
main(n_knn)

@time open("Test_Zero.xyz"; write=true) do f
    write(f, "$(size(X, 1))\n")
    write(f, "Initial Data ($(R_agg))\n")
    writedlm(f,hcat(X_f, X), ' ')
end

X = X |> cu
run(`paraview Test_Zero.xyz`)

# Fusioning tho Spheres

In [76]:
Size = Float32((findmax(X[:,1])[1] - findmin(X[:,1])[1])/2 + 1)
X_2 = vcat(X,X) |> cu
X_2[1:size(X,1),1] = X_2[1:size(X,1),1] .- (Size)
X_2[size(X,1):end,1] = X_2[size(X,1):end,1] .+ Size

747-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
  2.5
  9.0
 11.0
 13.0
  8.0
 10.0
 12.0
 14.0
  7.0
  9.0
 11.0
 13.0
  8.0
  ⋮
 14.0
  7.0
  9.0
 11.0
 13.0
  8.0
 10.0
 12.0
 14.0
  9.0
 11.0
 13.0

In [77]:
@time open("Test_Initial.xyz"; write=true) do f
    write(f, "$(size(X_2, 1))\n")
    write(f, "Initial Data ($(R_agg))\n")
    writedlm(f,hcat(X_f_2, X_2), ' ')
end

X = X_2 |> cu

run(`paraview Test_Initial.xyz`)

  0.084529 seconds (107.65 k allocations: 7.974 MiB, 62.84% compilation time)


Process(`paraview Test_Initial.xyz`, ProcessExited(0))

# Running with Two

In [78]:
# main(n_knn)

# Storing Data

In [79]:
# @time open("Test_Final.xyz"; write=true) do f
#     write(f, "$(size(X, 1))\n")
#     write(f, "Initial Data ($(R_agg))\n")
#     writedlm(f,hcat(X_f, X), ' ')
# end

In [80]:
# run(`paraview Test_Init.xyz Test_Final.xyz`)